<a href="https://colab.research.google.com/github/Maycon-douglas-Medeiros/MayContainGame/blob/master/Trabalho_IC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install pyswip
!apt-get install swi-prolog

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swi-prolog is already the newest version (8.4.2+dfsg-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [34]:
!pip install PyPDF2

In [35]:
!pip install reportlab

In [36]:
!apt-get install -y ttf-mscorefonts-installer
!fc-cache -fv

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ttf-mscorefonts-installer is already the newest version (3.8ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 2 dirs
/usr/share/fonts/X11: caching, new cache contents: 0 fonts, 2 dirs
/usr/share/fonts/X11/encodings: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/X11/encodings/large: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts/X11/util: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/msttcorefonts: caching, new cache contents: 0 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/

In [39]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pyswip import Prolog
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

credentials_file = '/service_account.json'

gc = gspread.service_account(filename=credentials_file)
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1fxeAGg3KmBJ3fm5fqJI0FvLKGkaDhRMxoGTt5jUtgss/edit?resourcekey=&gid=426825857#gid=426825857')
worksheet = sh.sheet1
data = worksheet.get_all_values()

respostas = []
for row in data[1:]:
    respostas.append(row[1:7])

with open('respostas.pl', 'w') as file:
    file.write("% Data\n")
    for linha in respostas:
        file.write("resposta({}).\n".format(','.join(linha[3:])))

prolog = Prolog()
prolog.consult('respostas.pl')

def create_temp_pdf_with_text(text, position):
    temp_pdf_path = "temp_text_pdf.pdf"
    c = canvas.Canvas(temp_pdf_path, pagesize=letter)

    lines = text.split('\n')

    for line in lines:
        c.drawString(position[0], position[1], line)
        position = (position[0], position[1] - 12)

    c.save()
    return temp_pdf_path

def add_text_to_pdf(input_pdf_path, output_pdf_path, text, line_number):
    reader = PdfReader(input_pdf_path)
    writer = PdfWriter()

    line_height = 12
    y_position = letter[1] - (line_number * line_height)
    position = (45.5, y_position)

    temp_pdf_path = create_temp_pdf_with_text(text, position)

    with open(temp_pdf_path, "rb") as temp_pdf_file:
        temp_reader = PdfReader(temp_pdf_file)
        temp_page = temp_reader.pages[0]

        for page_num in range(len(reader.pages)):
            original_page = reader.pages[page_num]
            original_page.merge_page(temp_page)
            writer.add_page(original_page)

    with open(output_pdf_path, 'wb') as output_pdf:
        writer.write(output_pdf)

input_pdf_path = 'esqueleto_laudo.pdf'
line_number = 27.2
contador = 0
for teste in prolog.query("resposta(X,Y,Z)"):
    if (teste['X'] == 'sim' and teste['Y'] == 'sim' and teste['Z'] == 'sim'):
        text_to_add = """
Exame realizado em mamógrafo digital, marca Siemens, nas incidências de rotina Craniocaudal (CC)
e Mediolateral Oblíqua (MLO) com extensão axilar usando foco de 0,3 mm, complementado com
tomossíntese para melhor visualização de achados anormais, adaptado para pacientes com implantes
mamários. Apesar de alguns desafios técnicos, como a posição do implante, os filmes serão impressos
e entregues posteriormente ao exame.
"""
    elif (teste['X'] == 'sim' and teste['Y'] == 'sim' and teste['Z'] == 'não'):
        text_to_add = """
Exame realizado em mamógrafo digital, marca Siemens, nas incidências de rotina Craniocaudal (CC)
e Mediolateral Oblíqua (MLO) com extensão axilar usando foco de 0,3 mm, complementado com
tomossíntese para melhor visualização de achados anormais, adaptado para pacientes com implantes
mamários. Os filmes serão impressos e entregues posteriormente ao exame.
"""
    elif (teste['X'] == 'sim' and teste['Y'] == 'não' and teste['Z'] == 'sim'):
        text_to_add = """
Exame realizado em mamógrafo digital, marca Siemens, nas incidências de rotina Craniocaudal (CC)
e Mediolateral Oblíqua (MLO) com extensão axilar usando foco de 0,3 mm, adaptado para pacientes
com implantes mamários. Apesar de alguns desafios técnicos, como a posição do implante, os filmes
serão impressos e entregues posteriormente ao exame.
"""
    elif (teste['X'] == 'não' and teste['Y'] == 'sim' and teste['Z'] == 'sim'):
        text_to_add = """
Exame realizado em mamógrafo digital, marca Siemens, nas incidências de rotina Craniocaudal (CC)
e Mediolateral Oblíqua (MLO) com extensão axilar usando foco de 0,3 mm, complementado com
tomossíntese para melhor visualização de achados anormais. Apesar de alguns desafios técnicos,
como a posição da paciente, os filmes serão impressos e entregues posteriormente ao exame.
"""
    elif (teste['X'] == 'sim' and teste['Y'] == 'não' and teste['Z'] == 'não'):
        text_to_add = """
Exame realizado em mamógrafo digital, marca Siemens, nas incidências de rotina Craniocaudal (CC)
e Mediolateral Oblíqua (MLO) com extensão axilar usando foco de 0,3 mm, adaptado para pacientes
com implantes mamários. Os filmes serão impressos e entregues posteriormente ao exame.
"""
    elif (teste['X'] == 'não' and teste['Y'] == 'sim' and teste['Z'] == 'não'):
        text_to_add = """
Exame realizado em mamógrafo digital, marca Siemens, nas incidências de rotina Craniocaudal (CC) e
Mediolateral Oblíqua (MLO) com extensão axilar usando foco de 0,3 mm, complementado com tomossíntese
para melhor visualização de achados anormais. Os filmes serão impressos e entregues posteriormente ao
exame.
"""
    elif (teste['X'] == 'não' and teste['Y'] == 'não' and teste['Z'] == 'sim'):
        text_to_add = """
Exame realizado em mamógrafo digital, marca Siemens, nas incidências de rotina Craniocaudal (CC)
e Mediolateral Oblíqua (MLO) com extensão axilar usando foco de 0,3 mm. Apesar de alguns desafios técnicos,
como a posição da paciente, os filmes serão impressos e entregues posteriormente ao exame.
"""
    else:
        text_to_add = """
Exame realizado em mamógrafo digital, marca Siemens, nas incidências de rotina Craniocaudal (CC)
e Mediolateral Oblíqua (MLO) com extensão axilar usando foco de 0,3 mm. Os filmes serão impressos
e entregues posteriormente ao exame.
"""

    output_pdf_path = f'esqueleto_laudo_novo_{contador}.pdf'
    add_text_to_pdf(input_pdf_path, output_pdf_path, text_to_add, line_number)
    contador += 1
